# 06-10-03

- KNN Clustering 
- Using Categories from KNN to get subset from information
- Continous Features(Only HighLevel Features)
    - MinMax Scaling
    - Remove Constant Features
    - Euclidean Distance
- Categorical Features
    - Hot Encoding
    - Jaccard

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns
from matplotlib import pyplot as plt 
import math
from ipynb.fs.full.Utils import createKDeplot, loadDataFrames, dropFeaturesWithNoVariance, loadCoverGroups
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy import stats
from ipynb.fs.full.Pipeline import removeConstantFeatures, applyBoxCoxAllFeatures, centerAndScale, applyEuclideanDistance, getIndexAndVector, applyBoxCoxFeatures,applyJaccard, getEuclideanJaccardDf 
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
transitionMatrices = pd.read_csv('./data/unified/07-transitionsClustered.csv')
transitionMatrices = transitionMatrices[['id','category']]

In [3]:
transitionMatrices

,id,category
0,23,1
1,105,0
2,118,2
3,120,1
4,292,2
...,...,...
5975,6002496,0
5976,6002497,1
5977,6002498,1
5978,6002499,1


In [4]:
cover = loadCoverGroups()
listCovers = cover[17]
originalSongId = listCovers[0]
coverSongId = listCovers[1]

In [5]:
originalSongId

6000764

In [6]:
df = pd.read_csv("./data/unified/05-allDataContinousCategoricalTransition.csv")
df = df.drop(columns=['Unnamed: 0', 'index'])

In [7]:
transitionMatrices

,id,category
0,23,1
1,105,0
2,118,2
3,120,1
4,292,2
...,...,...
5975,6002496,0
5976,6002497,1
5977,6002498,1
5978,6002499,1


In [8]:
df = pd.merge(df,transitionMatrices, on='id')

In [9]:
categoryOriginalSong = df[df.id == originalSongId].category.values[0]
df = df[df.category == categoryOriginalSong]

In [10]:
df

,id,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,...,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2,category
1,105,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344,0.185185,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0
5,407,0.577,0.890,0.0345,0.000013,0.0954,0.643,132.397,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
9,469,0.664,0.787,0.0239,0.038000,0.1930,0.585,102.952,0.007634,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
10,588,0.726,0.537,0.1140,0.316000,0.0707,0.882,96.889,0.101351,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0
12,635,0.550,0.145,0.0332,0.913000,0.1180,0.138,103.918,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5954,6002477,0.700,0.710,0.0532,0.327000,0.1290,0.582,167.971,0.000000,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
5964,6002488,0.760,0.838,0.0510,0.390000,0.1660,0.863,159.938,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0
5967,6002491,0.665,0.515,0.0442,0.454000,0.3670,0.805,93.082,0.000000,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0
5971,6002495,0.555,0.578,0.0277,0.113000,0.0747,0.333,83.382,0.000000,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0


In [11]:
df = df.drop(columns=['category'])

In [12]:
df

,id,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
1,105,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344,0.185185,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5,407,0.577,0.890,0.0345,0.000013,0.0954,0.643,132.397,0.000000,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,469,0.664,0.787,0.0239,0.038000,0.1930,0.585,102.952,0.007634,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10,588,0.726,0.537,0.1140,0.316000,0.0707,0.882,96.889,0.101351,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
12,635,0.550,0.145,0.0332,0.913000,0.1180,0.138,103.918,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5954,6002477,0.700,0.710,0.0532,0.327000,0.1290,0.582,167.971,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5964,6002488,0.760,0.838,0.0510,0.390000,0.1660,0.863,159.938,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
5967,6002491,0.665,0.515,0.0442,0.454000,0.3670,0.805,93.082,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
5971,6002495,0.555,0.578,0.0277,0.113000,0.0747,0.333,83.382,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
## Removing unamed and other unnecessary columns
df.reset_index(drop=True, inplace=True)
df = df.sort_values('id')
df.id = df.id.astype('int64')
# Saving idChords
ids = df.id
df = df.set_index('id')
df.reset_index(drop=True, inplace=True)

In [14]:
df

,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,2,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
0,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344,0.185185,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.577,0.890,0.0345,0.000013,0.0954,0.643,132.397,0.000000,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.664,0.787,0.0239,0.038000,0.1930,0.585,102.952,0.007634,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.726,0.537,0.1140,0.316000,0.0707,0.882,96.889,0.101351,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.550,0.145,0.0332,0.913000,0.1180,0.138,103.918,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2165,0.700,0.710,0.0532,0.327000,0.1290,0.582,167.971,0.000000,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2166,0.760,0.838,0.0510,0.390000,0.1660,0.863,159.938,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2167,0.665,0.515,0.0442,0.454000,0.3670,0.805,93.082,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2168,0.555,0.578,0.0277,0.113000,0.0747,0.333,83.382,0.000000,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [15]:
ids[ids ==originalSongId]

1769    6000764
Name: id, dtype: int64

## Pipeline

In [16]:
dfEuclidean, dfJaccard = getEuclideanJaccardDf(df)

In [17]:
dfEuclidean = dfEuclidean[['acousticness', 'danceability', 'energy', 'liveness', 'speechiness', 'tempo', 'valence']]

In [18]:
dfEuclidean

,acousticness,danceability,energy,liveness,speechiness,tempo,valence
0,0.000458,0.582,0.744,0.0405,0.0336,133.344,0.674
1,0.000013,0.577,0.890,0.0954,0.0345,132.397,0.643
2,0.038000,0.664,0.787,0.1930,0.0239,102.952,0.585
3,0.316000,0.726,0.537,0.0707,0.1140,96.889,0.882
4,0.913000,0.550,0.145,0.1180,0.0332,103.918,0.138
...,...,...,...,...,...,...,...
2165,0.327000,0.700,0.710,0.1290,0.0532,167.971,0.582
2166,0.390000,0.760,0.838,0.1660,0.0510,159.938,0.863
2167,0.454000,0.665,0.515,0.3670,0.0442,93.082,0.805
2168,0.113000,0.555,0.578,0.0747,0.0277,83.382,0.333


In [19]:
dfJaccard

,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2166,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2167,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2168,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [20]:
# dfEuclidean = applyBoxCoxFeatures(dfEuclidean, ['speechiness', 'acousticness', 'liveness', 'valence'])
dfEuclidean = dfEuclidean.copy()
dfEuclidean = removeConstantFeatures(dfEuclidean)
# dfEuclidean= applyBoxCoxFeatures(dfEuclidean, ['speechiness', 'acousticness', 'liveness', 'valence'])
# dfEuclidean= applyBoxCoxAllFeatures(dfEuclidean)
# dfEuclidean = centerAndScale(dfEuclidean)

In [21]:
scaler = MinMaxScaler()
scaler.fit(dfEuclidean)
dfEuclidean = pd.DataFrame(scaler.transform(dfEuclidean)).copy()

### Original Song & Cover Song

In [22]:
originalSongIndex,originalSongVector = getIndexAndVector(originalSongId, ids, dfEuclidean)
coverSongIndex,coverSongVector = getIndexAndVector(coverSongId, ids, dfEuclidean)

### Get Rankings Euclidean

In [23]:
ranksDFEuclidean  = applyEuclideanDistance(dfEuclidean, ids, originalSongVector)
ranksDFEuclidean

distance


,id,rank
0,6000764,0.000000
1,3461783,0.080215
2,1681054,0.121028
3,90551,0.165445
4,583331,0.175346
...,...,...
2165,1104539,1.526039
2166,6000782,1.527261
2167,975224,1.527261
2168,539502,1.582696


In [24]:
ranksDFEuclidean[ranksDFEuclidean.id == originalSongId]

,id,rank
0,6000764,0.0


In [25]:
ranksDFEuclidean[ranksDFEuclidean['id']==coverSongId]

,id,rank
476,6000667,0.732095


In [26]:
ranksDFEuclidean.loc[ranksDFEuclidean.id.isin(listCovers)]

,id,rank
0,6000764,0.000000
476,6000667,0.732095


### Get Rankings Jaccard

In [27]:
originalSongIndex,originalSongVector = getIndexAndVector(originalSongId, ids, dfJaccard)
coverSongIndex,coverSongVector = getIndexAndVector(coverSongId, ids, dfJaccard)

In [28]:
ranksDFJaccard  = applyJaccard(dfJaccard, ids, originalSongVector)

In [29]:
ranksDFJaccard

,id,rank
0,6000764,0.000000
1,6001308,0.333333
2,1005861,0.333333
3,872927,0.333333
4,152218,0.333333
...,...,...
2165,1128359,1.000000
2166,698704,1.000000
2167,1099865,1.000000
2168,127844,1.000000


In [30]:
ranksDFJaccard[ranksDFJaccard['id']==coverSongId]

,id,rank
136,6000667,0.571429


In [31]:
ranksDFJaccard[ranksDFJaccard['id']==coverSongId]

,id,rank
136,6000667,0.571429


### Cover songs

In [32]:
ranksDFJaccard.loc[ranksDFJaccard.id.isin(listCovers)]

,id,rank
0,6000764,0.000000
136,6000667,0.571429


<font color="red"> TODO: apply weight, or simply add rankings</font>

### Experiment to unify ranks

In [33]:
ranks = pd.merge(ranksDFJaccard,ranksDFEuclidean, on='id')

In [34]:
ranks['new_rank'] = ranks['rank_x'] + ranks['rank_y']

In [35]:
ranks = ranks.sort_values(by='new_rank')
ranks.reset_index(drop=True, inplace=True)

In [36]:
ranks.head(10)

,id,rank_x,rank_y,new_rank
0,6000764,0.000000,0.000000,0.000000
1,1005861,0.333333,0.286473,0.619806
2,1092970,0.333333,0.290818,0.624151
3,1090674,0.333333,0.321466,0.654799
4,273336,0.333333,0.387051,0.720385
5,277200,0.333333,0.392571,0.725905
6,90551,0.571429,0.165445,0.736873
7,1215658,0.333333,0.425271,0.758605
8,1715126,0.333333,0.432153,0.765486
9,1089370,0.571429,0.198455,0.769884


In [37]:
ranks[ranks['id']==coverSongId]

,id,rank_x,rank_y,new_rank
280,6000667,0.571429,0.732095,1.303524


In [38]:
ranks.loc[ranks.id.isin(listCovers)]

,id,rank_x,rank_y,new_rank
0,6000764,0.000000,0.000000,0.000000
280,6000667,0.571429,0.732095,1.303524
